In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import datetime

## Reading Data

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/auto+mpg
autompg = pd.read_csv('auto_mpg.csv')
autompg_disp = autompg['displacement'].astype(float)
autompg_mpg = autompg['mpg'].astype(float)
mean_disp = np.mean(autompg_disp)
min_disp = np.min(autompg_disp)
max_disp = np.max(autompg_disp)
print(mean_disp, max_disp, min_disp, max_disp-min_disp)
autompg_disp = autompg_disp.apply(lambda x:x/100)
print(autompg_disp.describe())
autompg_mpg = autompg_mpg.apply(lambda x: x/100)
print(autompg_mpg.describe())

In [ ]:
# A function to get a batch of data
def getbatch(xval, yval, arraylength, batchsize=30):
    count = 0 
    while count < arraylength/batchsize:
        randstart = random.randint(0, arraylength-batchsize-1)
        count += 1
        yield (xval[randstart:randstart+batchsize], yval[randstart:randstart+batchsize])

# Test
#for i in getbatch(train_X, train_Y, n_samples):
#    print(i)

## Setting hyper-parameters

In [ ]:
learning_rate = 0.003
n_epochs = 3000
display_step = 100
train_X = np.asarray(autompg_disp)
train_Y = np.asarray(autompg_mpg)
n_samples = train_X.shape[0]

## Defining X and Y as placeholders

In [ ]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

## Initializing Weights and Biases

In [ ]:
W = tf.Variable(np.random.randn())
b = tf.Variable(np.random.randn())

## Defining model

In [ ]:
# predicted is X*W+b. 
pred = tf.add(tf.multiply(X, W), b)

## Define cost, optimizer and accuracy

In [ ]:
# The cost function is ((predicted-actual)^2)/2*n_samples. 
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Add all ops that need to be initialized
# The initilization needs to be run only after session is created
# as in session.run(init) below.
init = tf.global_variables_initializer()

## Running the session 
This will run the graph and use all the tensors that were previously defined

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for (x, y) in getbatch(train_X, train_Y, n_samples):
            sess.run(optimizer, feed_dict={X: x, Y: y})
        if (epoch+1) % display_step == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            print("Epoch = {:04d} and cost = {:.9f}".format(epoch+1, c))

    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    finalW = sess.run(W)
    finalb = sess.run(b)
    print("The final W = %0.4f and b = %0.4f" %(finalW, finalb))